###### Environment initialization

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
keras = tf.keras

###### Dataset loading

In [ ]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

#split the data into 80% training, 10% testing and 10% validation
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

In [ ]:
get_label_name = metadata.features['label'].int2str

#display 2 images from datasets
for image, label in raw_train.take(2):
    plt.figure()
    plt.imshow(image)
    plt.title(get_label_name(label))

###### Data processing

In [1]:
#we need to convert our datasets image to the same size. Let resize to 160x160

IMG_SIZE = 160
def format_example(image, label):
    """
    returns an image that is reshapes to IMG_SIZE
    """
    image = tf.cast(image, tf.float32)
    image = (image/27.5) - 1
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

In [ ]:
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

In [ ]:
#let have a look of our image now
for image, label in raw_train.take(2):
    plt.figure()
    plt.imshow(image)
    plt.title(get_label_name(label))

In [ ]:
#finally we'll shuffle and batch our images
BATCH_SIZE = 32
SUFFLE_BUFFER_SIZE = 1000

train_batches = train.shuffle(SUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

###### Picking pretrainined model

The model we are goingto use as convolutionnal base for our model is the <b>MobileNet V2</b> developedat Google.
This model is trained on 1.4million images and as 1000 different classes.

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

#create  the base model from the pre_train model MobilNet V2
base_model = tf.keras.applications.MobilNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

In [ ]:
for image, _ in train_batches.take(1):
    pass

feature_batch = base_model(image)
print(feature_batch.shape)

###### Freezing the base

In [ ]:
base_model.trainable = False

In [ ]:
base_model.summary()

###### Adding our classifyer

In [7]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D

In [ ]:
#we'll add finally the prediction layer that w'll be a single dense  neuron - as we only have to predict two classes
prediction_layer = keras.layers.Dense(1)

In [ ]:
#let combineright now all these layers togetter 
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])

In [ ]:
model.summary()

###### Model training

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])

In [ ]:
#wecan evaluate the model right now to see how it does before training it on our new images
initial_epochs = 3
validation_steps = 20

loss0, accuracy0 = model.evaluate(validation_batches, steps = validation_steps)

In [ ]:
#Now we can train it on our image
history = model.fit(train_batches, epochs=initial_epochs, validation_data=validation_batches)
acc = history.history['accuracy']
print(acc)

###### We can save model

In [ ]:
model.save('dogs_vs_cats.h5')
#if wanna load
new_model = tf.keras.models.load_model('dogs_vs_cats.h5')